#### **Install Dependencies**

**Install Pytorch**: Go to `pytorch.org`, configure your settings (in my case, I choose- stable,windows,pip,python,cpu), copy the run command and run it in a python shell.

In [2]:
!pip3 install torch torchvision torchaudio
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118     # gpu version

**Install Other Dependencies**

In [4]:
#pip install transformers requests beautifulsoup4 pandas numpy

* `transformers`: transformer is the library of HuggingFace, from where we will load the model "bert-base-multilingual-uncased-sentiment" in order to caculate sentiment score.
> `bert-base-multilingual-uncased-sentiment`: This a bert-base-multilingual-uncased model finetuned for sentiment analysis on product reviews in six languages: English, Dutch, German, French, Spanish and Italian. It predicts the sentiment of the review as a number of stars (between 1 and 5). [`For more details click here`](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment)
* `requests`: is for sending request to a website for extracting data from the website
* `beautifulsoup`: to fetch the required data from the site
* `pandas`: to represent the data to dataframe for better outlooking
* `numpy`: to convert data into numeric for further use in model 

In [6]:
# Import modules
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

#### **Instantiate Model**

#### **Encode and Calculate Sentiment Score**

#### **Collect Data form Online**

#### **Load Data into DataFrame**

#### **Calculate Sentiment Score**